# Calculating REE concentrations in zircon
This notebook allows you to calculate the basic concentration of REE in Zircon following Blereau et all 2020. 

Inputs:
There are currently no inputs to this notebook, though you may wish to ammend the equilibrium concentrations of REEs that are used, as well as the location, size and resolution of spot analysis

If you wish to save any changes you will need to save this notebook to your own computer.



# New to Jupyter Notebooks?
A Jupyter Notebook is a way of combining text and code (here we use the programming language Python3), and running that code in a web browser. Remember '#' is used to provide comments within the code cells

Some useful links for help with jupyter notebooks:

https://towardsdatascience.com/jypyter-notebook-shortcuts-bf0101a98330

#### Jupyter usage
| operator | name  |
|:------|:---------------|
|   ctrl + enter  | run a cell |
|   up/down arrows  | move between cells |

# OK, let's start....
# DO NOT change this section
Please just run the cells as they are. To run code cells, select and press shift + enter

## Code imports
Let's start with code imports. To run code cells, select and press shift + enter

In [1]:
import math
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import scipy.special as erf
import pandas as pd
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import FileLinks, FileLink, DisplayObject
import copy
from tabulate import tabulate



Bad key "text.kerning_factor" on line 4 in
/home/ellie/anaconda3/envs/py3/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


## Function definitions

In [2]:
def diffusion_profile(r, t, a, C0, D, siy):
    """diffusion profile for REE in zircon for instantaneous 
    spherical source diffuse throughout the surrounding medium

    inputs are:
        r = radius of diffusion
        t = time in years (1 yr = 31,556,926 s = siy)
        C0 = uniform concentration in initial sphere (initial)
        a = radius of sphere
        D= diffusion coefficient
    outputs are:
        C = Concentration
        C_change = change in concentration
    """
    #instantaneous spherical source diffuse throughout the surrounding medium   
    C = C0*((0.5) *  (erf.erf((a+r)/(2*(np.sqrt(D*t*siy)))) + erf.erf((a-r)/(2*(np.sqrt(D*t*siy)))))  - 1/r *np.sqrt((D*t*siy)/np.pi) * (np.exp(-(a-r)**2/(4*D*t*siy))-np.exp(-(a+r)**2/(4*D*t*siy)))) # not normalised
    
    # concentration change from initial value
    C_change = C / C0
    
    return [C, C_change]

def sectorarea(r, h1, h2=[]):
    """ returns the area of a segment for circle of radius r, 
    where h1 is the distance from centre to inner bounding chord
          h2 is the distance from centre to outer bounding chord
    if h2 is defined, the area between two chords at radius h1 and h2 will be given
    """
    if h2 == []:
        h2 = r
    area_inner = r * (r * np.arccos( h1 / r) - h1 * np.sqrt( 1 - (h1**2 / r**2)))
    area_outer = r * (r * np.arccos( h2 / r) - h2 * np.sqrt( 1 - (h2**2 / r**2)))
    
    return area_inner - area_outer 

## Input tables

The below tables provide some key information for a given temperature about a selection of rare Earth elements ("Gd", "Tb", "Dy", "Ho", "Er", "Tm", "Yb", "Lu"). This includes:

- activation energies (and errors) 
- initial concentrations we use
- equilibrium conentrations
- pre-exponential factor (D0, used to calculate diffusion coefficients)

Note: pre-exponential factor for Dy, Yb and Sm are given in Cherniak et al., 1997. We use a polynomial fit

$y=436.28x^2 - 859.18x+427.58$

to calculate D0 for other REE elements used here. See Cherniak et al. 1997 for further methodology.

In [3]:
elements = ["Gd", "Tb", "Dy", "Ho", "Er", "Tm", "Yb", "Lu"]
temperature = [850, 900, 950, 1000, 1050, 1100]

base_table = pd.DataFrame([[4117660.35, 780.17, 40.63,  21.626, 19.66],
                           [818932.3, 755.28, 36.4, 5.082, 3.63],
                           [228722.09, 734.1, 34.73,  50.967, 24.27],
                           [95249.56, 717.85, 35.45, 17.792, 5.56],
                           [55017.45, 705.74, 38.01,  69.916, 15.89],
                           [41250.2, 697.04, 41.93,  14.52,  2.42],
                           [37797.49, 691.09, 46.74,126.75, 16.25],
                           [40091.48, 687.3, 52.05,  24.3, 2.43]],
                          columns=[ "D0", 
                                    "activation_energy", 
                                    "activation_energy_error",
                                    "initial_concentration", 
                                    "Equilib_conc"], 
                          index=elements)

tables = []

for T in temperature:
    temp_table = copy.deepcopy(base_table)
    temp_table["T_celcius"] = np.ones(len(elements)) * T
    tables.append(temp_table)


Here we add the diffusion coefficients to the tables calculated based on the equation within Cherniak et al., 1997 for the relevant temperatures.

Note: we add the min and max possible diffusion coefficients to the table, but use the average diffusion coefficient in our work.

In [4]:
for t in tables:
    temp = []
    for i in range(len(t)):
        temp.append(t['T_celcius'][i]+273.15)
    t['T_Kelvin']=temp
    
for t in tables:
    temp = []
    temp_p = []
    temp_n = []
    for i in range(len(t)):
        D0 = t['D0'][i]
        Ea = t['activation_energy'][i]
        Ea_err = t['activation_energy_error'][i]
        T = t['T_Kelvin'][i]
        temp.append(D0 * np.exp(-Ea/( 0.00831447 * T)))
        temp_p.append(D0 * np.exp(-(Ea-Ea_err)/( 0.00831447 * T)))
        temp_n.append(D0 * np.exp(-(Ea+Ea_err)/( 0.00831447 * T)))
    t['diffusion_coefficient'] = temp
    t['diffusion_coefficient_max'] = temp_p
    t['diffusion_coefficient_min'] = temp_n
    

## variables - you may change these

times: time in years (1 yr = 31,556,926 s = siy) to be analysed

siy: conversion factor of years to seconds - 31556926 seconds in a year 

a: radius chosen as grain boundary of the theoritical spherical grain

r: radius to be analysed (in metres)

C_eq: equilibrium concentration (taken from input table)

Normalised_C_eq: Equilibrium normalisation concentration (we set to 110 for all elements)

Norm_values: Chondrite normalisation values from Anders and Grevesse (1989)

<!-- C0: (initial) uniform concentration in sphere -->

<!-- C: concentration at radius (r) at a time (t)

D: Diffusion coefficient (middle and maximum) m^2s
 -->

In [5]:
# time scales chosen are 0.5, 1, 5, 10, 30, 100, 200 My
times = [500000, 1000000, 5000000, 10000000, 30000000, 100000000, 200000000]

# conversion factor of years to seconds - 31556926 seconds in a year 
siy = 365.2425 * 24*60*60

# radius chosen as grain boundary is 50 microns (given in metres: 50 microns/10000=0.005 cm 0.00005 m)
a = 0.00005

# radii under investigation up to rim is 5-50 microns
radii = np.arange(0.0000005,0.00005, 0.0000005) 

#equilibrium concentration (taken from input table)
C_eq = base_table["Equilib_conc"]

Normalised_C0 = [110, 140, 210, 320, 440, 600, 780, 1000]
Normalised_C_eq = [110, 110, 110, 110, 110, 110, 110, 110]

# from Anders and Grevesse - chondrite normalisation values
Norm_values = [0.1966, 0.0363, 0.2427, 0.0556, 0.1589, 0.0242, 0.1625, 0.0243]

## Run function on all timesteps / radii 
These concentration values are saved in the tables labeled "tables_conc", and are used to produce Figure 9.

In [6]:
# this makes a table of tables
tables_conc = []

for t in range(len(tables)):
    D = tables[t]['diffusion_coefficient']
    Dp = tables[t]['diffusion_coefficient_max']
    Dn = tables[t]['diffusion_coefficient_min']
    C0 = tables[t]['initial_concentration'] - tables[t]['Equilib_conc']
    temp_conc_table = np.zeros((len(radii), len(times), len(elements)))
    for i in range(len(radii)):
        for j in range(len(times)):
            conc = diffusion_profile(radii[i], times[j], a, C0, D, siy)
            temp_conc_table[i][j][:] = conc[0]
    
    tables_conc.append(temp_conc_table)


# Calculating theoretical analytical spot concentrations

Here we calculate the concentration of a theoretical SIMS/LA-ICP-MS spots.

We use the example of a 20 micron spot that has a 10 micron radius. This is done by taking the concentration over 1 micron segments and summing them across the spot.

The output tables are a n-d array of element concentrations with distance.

r: spot radius = 10 microns

d: distance from centre of grain to place centre of spot. Default assumes a 50 micron grain, with spot entred at 40 microns (10 microns from rim)

res: resolution = 1 micron (segment width)
    

In [7]:
d = 40 
r = 10 
res = 1

In [8]:
def spot_fun(d, r, res, t, e, conc_table, normalise=True):
    
    
    temp_table = np.zeros((len(t), len(e)))
    
    # number of area segments to get desired resolution
    segs = int(r * 2 / res)
    
    # what radial distances 
    dist = [(d - r + i) for i in range(1, segs+1) ]

    dist_micron = [(d - r + i) *(1e-6) for i in range(1, segs+1) ]
    spot_seg_radii_idx = []

    for i in dist_micron:
        spot_seg_radii_idx.append((np.abs(i - radii)).argmin())


    norm_area = np.pi * r**2

    for j in range(len(t)):
        temp_conc = conc_table[:, j, :]

        for i in range(len(e)):
            for k in range(len(dist)):
                r_conc = temp_conc[spot_seg_radii_idx[k]][i] 
                norm_seg_area = sectorarea(r, r-k-1, r-k) / norm_area
                temp_table[j][i] += (r_conc+C_eq[i]) * norm_seg_area
                
            if normalise:
                temp_table[j][i] = temp_table[j][i] / Norm_values[i]
                if temp_table[j][i] < Normalised_C_eq[i]:
                    temp_table[j][i] = Normalised_C_eq[i]
                
    return temp_table



tables_spot_outer = []
tables_spot_10 = []

for t in range(len(tables_conc)):
    tables_spot_outer.append(spot_fun(d, r, res, times, elements, tables_conc[t]))
    tables_spot_10.append(spot_fun(10, r, res, times, elements, tables_conc[t]))


# output tables for figure 10

In [9]:
print(f"These tables are the concentration of REEs expected at a radial distance {d} microns")
print(f"with a spot of radius {r}, for a given temperature in degrees")

These tables are the concentration of REEs expected at a radial distance 40 microns
with a spot of radius 10, for a given temperature in degrees


In [10]:
elements = ["Gd", "Tb", "Dy", "Ho", "Er", "Tm", "Yb", "Lu"]
time_labels = ["0.5My", "1.0My", "5.0My", "10.0My", "30.0My", "100.0My", "200.0My"]

for t in range(len(tables_spot_outer)):
    print(f"\nfor temperature = {temperature[t]}")
    display(pd.DataFrame(tables_spot_outer[t].transpose(),
                         columns=time_labels, 
                         index=elements))


for temperature = 850


,0.5My,1.0My,5.0My,10.0My,30.0My,100.0My,200.0My
Gd,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000
Tb,139.626056,139.626022,139.625877,139.625768,139.625496,139.624965,139.624256
Dy,208.971508,208.971353,208.970697,208.970206,208.968978,208.962868,208.910425
Ho,317.942611,317.942132,317.940112,317.938599,317.933894,317.759443,317.312453
Er,436.819588,436.818513,436.813976,436.810504,436.734470,435.769244,434.437562
Tm,595.321475,595.319294,595.310025,595.290051,594.706347,591.996587,588.831994
Yb,773.634940,773.631035,773.606807,773.414876,771.568912,765.888114,759.479569
Lu,991.572392,991.565872,991.430846,990.645607,986.707685,976.114395,964.257619



for temperature = 900


,0.5My,1.0My,5.0My,10.0My,30.0My,100.0My,200.0My
Gd,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000
Tb,139.625674,139.625481,139.624634,139.621233,139.554771,139.299703,139.007171
Dy,208.969880,208.969051,208.947576,208.836436,208.320112,206.947570,205.412165
Ho,317.937821,317.935001,317.618269,317.030974,315.117777,310.259875,304.830652
Er,436.808644,436.770321,435.412320,433.832024,428.936269,416.563696,402.798607
Tm,595.267653,594.937316,591.299851,587.678203,576.548881,548.479841,518.080314
Yb,773.293907,772.204614,764.692435,757.508746,735.443486,680.475852,624.953120
Lu,990.342430,988.016364,974.155747,961.029510,920.727978,823.694616,735.204742



for temperature = 950


...

,0.5My,1.0My,5.0My,10.0My,30.0My,100.0My,200.0My
Gd,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000
Tb,138.957018,138.442102,135.580408,132.987402,126.729882,118.021761,112.405558
Dy,205.631234,203.241783,190.309194,180.035839,159.538588,133.389476,118.945465
Ho,306.778216,299.104563,261.203532,236.350932,191.261958,139.139518,118.652284
Er,409.758092,391.827928,316.459592,274.208552,199.711516,132.312641,113.700910
Tm,536.330016,499.697071,370.962238,304.363026,195.379082,124.768775,110.000000
Yb,661.370422,598.850447,407.376861,311.951363,180.460135,117.979963,110.000000
Lu,795.961485,700.838939,431.609001,305.709530,165.828767,113.366179,110.000000



for temperature = 1050


,0.5My,1.0My,5.0My,10.0My,30.0My,100.0My,200.0My
Gd,110.000000,110.000000,110.000000,110.000000,110.0,110.0,110.0
Tb,134.243264,131.017586,120.318968,114.867024,110.0,110.0,110.0
Dy,185.942490,174.507379,141.858793,126.389669,110.0,110.0,110.0
Ho,252.298971,226.251162,156.743889,130.216490,110.0,110.0,110.0
Er,303.660666,260.197559,153.913437,125.012154,110.0,110.0,110.0
Tm,353.705684,285.531813,146.088469,119.460473,110.0,110.0,110.0
Yb,385.284824,289.158880,136.087535,114.354023,110.0,110.0,110.0
Lu,403.706000,280.607321,128.181399,110.800883,110.0,110.0,110.0



for temperature = 1100


,0.5My,1.0My,5.0My,10.0My,30.0My,100.0My,200.0My
Gd,110.000000,110.000000,110.0,110.0,110.0,110.0,110.0
Tb,118.816219,113.246309,110.0,110.0,110.0,110.0,110.0
Dy,138.995997,123.738093,110.0,110.0,110.0,110.0,110.0
Ho,153.575479,127.963716,110.0,110.0,110.0,110.0,110.0
Er,152.151419,124.020723,110.0,110.0,110.0,110.0,110.0
Tm,145.847927,119.345262,110.0,110.0,110.0,110.0,110.0
Yb,136.789303,114.657529,110.0,110.0,110.0,110.0,110.0
Lu,129.247460,111.233605,110.0,110.0,110.0,110.0,110.0


# Have a play with plotting parameters

again, just use "shift" + "enter" on the keyboard to run the cell. This will produce a figure with drop down menu options. Use these to investigate parameters rather than changing the code directly.


In [11]:
%matplotlib inline

In [12]:
plt.close()
plt.rcParams['figure.figsize'] = [7, 7]

## if you would like to save figures, set the below to True. The figures will save in the binder home page where you will be able to download them

In [13]:
save_figs = False 

## Figure 9

In [15]:
plt.close()
def plot_concs(Element, Temperature, Time_Scale): #C/Co v x distribution
    plt.figure()
    plt.xlabel="Radius (microns)"
    plt.ylabel="C/C0 (ppm)"
    try:
        El_ix = elements.index(Element)
        e_range=1
    except ValueError:
        e_range=len(elements)
        legend_labels = elements
        
    try:      
        Tp_ix = temperature.index(float(Temperature))
        T_range = 1
        
    except ValueError:
        T_range=len(tables)
        legend_labels = ["850", "900", "950", "1000", "1050", "1100"]
        
    try:
        TS_ix = time_labels.index(Time_Scale)
        t_range=1
    except ValueError:
        t_range = len(times)
        legend_labels = [i/1e6 for i in times]
    

    for T in range(T_range):
        if T_range==1:
            T = Tp_ix

        for t in range(t_range):
            if t_range==1:
                t = TS_ix
            for e in range(e_range):
                if e_range==1:
                    e = El_ix
                temp_conc = tables_conc[T][:, t, e] + tables[T]["Equilib_conc"][e]
                #1000000 m to microns
                plt.plot(radii*1000000, temp_conc)


    try:
        plt.legend(legend_labels)
    except UnboundLocalError:
        pass

    if save_figs == True:
        outname = f"{Element}_{Temperature}_{Time_Scale}_normalised_spot_concentrations.png"  
        plt.savefig(outname)#, dpi=None)
                
interact(plot_concs, Element=["Gd", "Tb", "Dy", "Ho", "Er", "Tm", "Yb", "Lu", "all"], Temperature= ["850", "900", "950", "1000", "1050", "1100", "all"], Time_Scale=["0.5My", "1.0My", "5.0My", "10.0My", "30.0My", "40.0My", "100.0My", "150.0My", "200.0My", "all"])

interactive(children=(Dropdown(description='Element', options=('Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu',…

<function __main__.plot_concs(Element, Temperature, Time_Scale)>

## Figure 10

In [17]:
plt.close()
def plot_concs(Temperature, Time_Scale): #C/Co v x distribution  (Element, Temperature, Time_Scale)
    plt.figure()
    try:      
        Tp_ix = temperature.index(float(Temperature))
        T_range = 1
        
    except ValueError:
        T_range=len(tables)
        legend_labels = ["850", "900", "950", "1000", "1050", "1100"]
        
    try:
        TS_ix = time_labels.index(Time_Scale)
        t_range=1
    except ValueError:
        t_range = len(times)
        legend_labels=([i/1e6 for i in times])
    

    for T in range(T_range):
        if T_range==1:
            T = Tp_ix
        for t in range(t_range):
            if t_range==1:
                t = TS_ix
            
            temp_spot = tables_spot_outer[T][t, :]
            plt.plot(elements, temp_spot)

    
    try:
        plt.legend(legend_labels)
    except:
        pass
   
    plt.xticks(np.arange(len(elements)),elements)
    plt.plot(elements, Normalised_C_eq, color='k', label='C_eq')
    plt.plot(elements, Normalised_C0, color='grey', label='C_o')

    plt.ylabel="REE/Chondrite (ppm)"
    plt.yscale("log")
    
    if save_figs == True:
        outname = f"{Temperature}_{Time_Scale}_normalised_spot_concentrations.png"  
        plt.savefig(outname)#, dpi=None)
                
interact(plot_concs, Temperature= ["850", "900", "950", "1000", "1050", "1100", "all"], Time_Scale=["0.5My", "1.0My", "5.0My", "10.0My", "30.0My", "100.0My", "200.0My", "all"]) 

interactive(children=(Dropdown(description='Temperature', options=('850', '900', '950', '1000', '1050', '1100'…

<function __main__.plot_concs(Temperature, Time_Scale)>

## Figure 11

In [18]:
plt.close()
def plot_concs(Temperature, Time_Scale): #C/Co v x distribution  (Element, Temperature, Time_Scale)
    plt.figure()
    try:      
        Tp_ix = temperature.index(float(Temperature))
        T_range = 1
        
    except ValueError:
        T_range=len(tables)
        legend_labels = ["850", "900", "950", "1000", "1050", "1100"]
        
    try:
        TS_ix = time_labels.index(Time_Scale)
        t_range=1
    except ValueError:
        t_range = len(times)
        legend_labels=([i/1e6 for i in times])
    

    for T in range(T_range):
        if T_range==1:
            T = Tp_ix
        for t in range(t_range):
            if t_range==1:
                t = TS_ix
            
            temp_spot_outer = tables_spot_outer[T][t, :]
            temp_spot_inner = tables_spot_10[T][t, :]
            plt.fill_between(elements, temp_spot_outer, temp_spot_inner)
#             plt.plot(elements, tables_spot_outer)

    
    try:
        plt.legend(legend_labels)
    except:
        pass
   
    plt.xticks(np.arange(len(elements)),elements)
    plt.plot(elements, Normalised_C_eq, color='k', label='C_eq')
    plt.plot(elements, Normalised_C0, color='grey', label='C_o')

    plt.ylabel="REE/Chondrite (ppm)"
    plt.yscale("log")
    
    if save_figs == True:
        outname = f"{Temperature}_{Time_Scale}_normalised_spot_concentrations.png"  
        plt.savefig(outname)#, dpi=None)
                
interact(plot_concs, Temperature= ["850", "900", "950", "1000", "1050", "1100", "all"], Time_Scale=["0.5My", "1.0My", "5.0My", "10.0My", "30.0My",  "100.0My", "200.0My", "all"]) 



interactive(children=(Dropdown(description='Temperature', options=('850', '900', '950', '1000', '1050', '1100'…

<function __main__.plot_concs(Temperature, Time_Scale)>